### Test dash-extentions Server side caching

In [2]:
# new Cached dash_extensions
import datetime
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import numpy as np
import pandas as pd
from dash_extensions.enrich import Dash, ServersideOutput, Output, Input, State, Trigger

ROWS_FOR_DASHTABLE=1000

def _make_df(value):
    dict_df = {'rownum':list(range(1,value+1))}
    for i in range(1,11):
        d = np.random.rand(int(value))
        dict_df[f'c{i}']=d
    df = pd.DataFrame(dict_df)
    return df


def _make_dt(dt_id,df,displayed_rows=100,page_action='native'):
    dt = dash_table.DataTable(
        id=dt_id,
        page_current= 0,
        page_size=displayed_rows,
        page_action=page_action, 
    )
    dt.data=df.to_dict('rows')
    dt.columns=[{"name": i, "id": i} for i in df.columns.values]                    
    return dt

# Drop down options.
options = [{"label": x, "value": x} for x in [1, 10, 100, 1000, 10000, 100000, 1000000, 10000000, 100000000]]
# Create app.
url_base_pathname='/app8812/'
app = Dash(prevent_initial_callbacks=True,url_base_pathname=url_base_pathname)
app.title='test_dash_extensions_caching'
server = app.server
dt_data = _make_dt(
    'dt_data',pd.DataFrame(),
    displayed_rows=ROWS_FOR_DASHTABLE,page_action='custom'
)

dt_data_div = html.Div([dt_data],id='dt_data_div')

app.layout = html.Div([
    html.Button("Run benchmark (with cache)", id="btn"), dcc.Dropdown(id="dd", options=options, value=1),
    dcc.Store(id="time"), 
    dcc.Loading(children=[dcc.Store(id="store"),dt_data_div], 
                fullscreen=True, type="dot"), html.Div(id="log")
])


@app.callback([ServersideOutput("store", "data"), ServersideOutput("time", "data")],
              Trigger("btn", "n_clicks"), State("dd", "value"))
def query(value):
    df = _make_df(value)
    return df, datetime.datetime.now()

# @app.callback([Output("log", "children"),Output('dt_data_div','children')], Input("store", "data"), State("time", "data"))
@app.callback([Output("log", "children"),Output('dt_data','data'),
               Output('dt_data','columns'),Output('dt_data','page_count')], 
              [Input("store", "data"),Input('dt_data','page_current'),Input('dt_data','page_size')], 
              State("time", "data"))
def calc(df, page_current,page_size,time):
    toc = datetime.datetime.now()
    elapsed_txt = "ELAPSED = {}s".format((toc - time).total_seconds())
    pagcur = page_current
    if (pagcur is None) or (pagcur<0):
        pagcur = 0
    ps = page_size
    if (ps is None) or (ps<1):
        ps = ROWS_FOR_DASHTABLE
    beg_row = page_current*page_size
    if pagcur*ps > len(df):
        beg_row = len(df) - ps

    dict_data = df.iloc[beg_row:beg_row + ps].to_dict('records')
    cols = [{"name": i, "id": i} for i in df.columns.values]
    page_count = int(len(df)/ps) + (1 if len(df) % ps > 0 else 0)
    return elapsed_txt,dict_data,cols,page_count


if __name__ == '__main__':
    app.run_server(port=8812)
    